<a href="https://colab.research.google.com/github/aneesh-shetye/NLP/blob/master/seq_2_seq_with_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

installing the required versions of libraries

In [1]:
!pip install torchtext==0.6.0

     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 1.2MB 21.3MB/s 
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1


In [2]:
!python -m spacy download de
!python -m spacy download en 

     |████████████████████████████████| 14.9MB 18.7MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-cp37-none-any.whl size=14907057 sha256=b63ff178d8557a49e86795169b6c429f692fd5de470f037d5b3fc8338b6418ef
  Stored in directory: /tmp/pip-ephem-wheel-cache-qgrmrtui/wheels/ba/3f/ed/d4aa8e45e7191b7f32db4bfad565e7da1edbf05c916ca7a1ca
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
import torch
import  torchtext
from torchtext.data.utils import  get_tokenizer
from torchtext.vocab import Vocab 
import torch.nn as nn
import numpy as np 
import spacy 
import random 
from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator
from torch import Tensor 

In [4]:
en_tokenizer = torch.device('cpu')

if torch.cuda.is_available(): 
    device = torch.device("cuda")
    print("You are using ", torch.cuda.get_device_name())
else: 
    print("You are using cpu")
    

You are using  Tesla T4


In [5]:
#defing tokenizer: the things that split your sentence into words or subwords
en_tokenizer = get_tokenizer('spacy', language='en')
de_tokenizer = get_tokenizer('spacy', language='de')

In [6]:
src_field = Field(init_token = '<src_sos>', eos_token='<src_eos>', lower= True, tokenize= de_tokenizer)
trg_field = Field(init_token = '<trg_sos>', eos_token = '<trg_eos>', lower=True, tokenize=en_tokenizer)

train_data, valid_data, test_data = Multi30k.splits(exts=('.de', '.en'), fields=(src_field, trg_field))

#making the vocabulary : 

src_field.build_vocab(train_data, min_freq =2) # replaces words which appear less than twice with <unk> token
trg_field.build_vocab(train_data, min_freq=2)

print("Source vocab size is :", len(src_field.vocab))
print("Target vocab size is : ", len(trg_field.vocab))



downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 1.04MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 278kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 264kB/s]


Source vocab size is : 7855
Target vocab size is :  5893


making train and test iterators: 

In [7]:
BATCH_SIZE = 128

train_iterator = torchtext.data.BucketIterator(train_data, batch_size=BATCH_SIZE, device=device)
test_iterator = torchtext.data.BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

In [8]:
"""
defining the encoder: 

    input : src_sentence
    output : hidden_states + 
             final hidden states (encoding)

"""

class Encoder_RNN(nn.Module): 
    
    def __init__(self, 
                 src_dim : int, 
                 no_features_enc: int, 
                 hidden_size: int, 
                 no_layers_enc: int,
                 dropout: float = 0.5): 

        super().__init__()

        self.embed = nn.Embedding(src_dim , no_features_enc)
        self.recurrent = nn.GRU(no_features_enc, hidden_size, bidirectional=True, num_layers = no_layers_enc) 
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_size*2, hidden_size)


    def forward(self, 
                src: Tensor)-> tuple : 

                #src.shape = [seq_len, batch_size]
                src = src.to(device)

                embeddings = self.dropout(self.embed(src))
                #embeddings.shape = [seq_len, batch_size, no_features_enc]

                h, h_n = self.recurrent(embeddings) #h_n.shape  [num_layers*2, batch_size, hidden_size ]

                h_n = torch.cat((h_n[-2, : , : ], h_n[-1, : , :]), dim=1) # given in the paper that we conccatenate the forward and backward states

                #h_n.shape = [batch_size, hidden_size*2]
                #h.shape = [src_len, batch_size, hidden_size_enc*2]
                
                h_n = self.linear(h_n)
                #h_n.shape = batch_size, dec_hidden_size

                h_n = h_n.unsqueeze(0)
                #h_n.shape = 1, batch_Size, dec_hidden_size
                
                return h, h_n

In [9]:
encoder = Encoder_RNN(src_dim=200, 
                      no_features_enc=100, 
                      hidden_size=200, 
                      no_layers_enc=3)
encoder.to(device)
a = torch.zeros([38, 64], dtype=torch.long)
h, h_n = encoder(a)


In [10]:
""" 
defining attention (as used to get the results in paper): 

    input : enc_hidden_states 
            dec_hidden_state
    output : context vector 
"""

class Attention(nn.Module):

    def __init__(self, 
                 hidden_size: int): 
        
        super().__init__()

        #enc and dec hidden size are equal 
        self.linear_enc = nn.Linear(hidden_size*2, hidden_size, bias=False)
        self.linear_dec = nn.Linear(hidden_size, hidden_size, bias=False)

        self.linear_sum = nn.Linear(hidden_size,1, bias=False )
        self.softmax = nn.Softmax(dim=0)


    def forward(self, 
                h: Tensor, 
                s_prev: Tensor)-> Tensor : 

        
        #h.shape = src_len , batch_size, hidden_size*2
        #s_prev.shape = batch_size, hidden_size 

        h, s_prev = h.to(device), s_prev.to(device)

        h_2 = self.linear_enc(h) #h.shape = src_len, batch_size, hidden_size
        s_prev = self.linear_dec(s_prev) #s_prev.shape = batch_size, hidden_size

        sum = torch.tanh(h_2 + s_prev) #sum.sahpe = src_len, batch_size, hidden_size

        a = self.linear_sum(sum) #a.shape = src_len, batch_size, 1

        alpha = self.softmax(a)  #a.shape = src_len, batch_Size, 1

        c = torch.mul(alpha, h) #c.shape = src_len, batch_size, hidden_size*2

        c = torch.sum(c, 0) #c.shape = batch_size, hidden_size*2


        return c



In [11]:

"""
Decoder : 
    input: s_prev, 
            a,
            prev_output 

    output: s, 
            output 


"""


class Decoder_RNN(nn.Module): 

    def __init__(self, 
                 hidden_size: int, 
                 dropout:float, 
                 no_features_dec: int,
                 output_dim: int): 

        super().__init__()

        self.embed = nn.Embedding(output_dim, no_features_dec)
        self.recurrent = nn.GRU(no_features_dec + 2*hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_dim)
        self.dropout = nn.Dropout(dropout)


    def forward(self, 
                prev_output: Tensor, 
                s_prev: Tensor, 
                c: Tensor)-> tuple : 

        # prev_output.shape - [1, batch_size]
        #s_prev.shape = [1, batch_size, dec_hidden_size]
        #c.shape = [batch_size, hidden_size*2]

        prev_output, s_prev, c = prev_output.to(device), s_prev.to(device), c.to(device)

        c = c.unsqueeze(0)

        embeddings = self.dropout(self.embed(prev_output))
        #embeddings.shape = [1, batch_size, no_features_dec]

        embeddings = embeddings.unsqueeze(0)

        output , s = self.recurrent(torch.cat((embeddings, c), dim=2), s_prev) 
        #output.shape = [1, batch_size, hidden_size]
        #s.shape = [1, batch_size, hidden_size]

        output = self.linear(output)
        #output.shape = [1, batch_size, output_dim ]

        return output, s


In [12]:
"""

Seq2Seq with attention model : 
    input : enc_hidden_states
            prev_output,
            s_prev
    
    output : output,  
             s

"""

class Seq_2_Seq_att(nn.Module): 

    def __init__(self, 
                 dec: nn.Module, 
                 att: nn.Module): 

        super().__init__()

        self.dec = dec 
        self.att = att
        
    
    def forward(self, 
                h: Tensor,
                prev_out: Tensor, 
                s_prev: Tensor)-> tuple: 

        h, prev_out, s_prev = h.to(device), prev_out.to(device), s_prev.to(device)

        # print(h.shape)
        c = self.att(h, s_prev)
        output, s = dec(prev_out, s_prev, c)

        return output, s       




In [13]:
"""
training loop : 
tf = 0 for testing
"""

def train(iterator: torchtext.data.BucketIterator, 
          model: nn.Module,
          enc: nn.Module,
          criterion: nn.Module, 
          tf: float = 0.9):
    
    model.train()

    output_dim = len(trg_field.vocab)


    for i, data in enumerate(iterator): 

        batch_loss = 0
        data.src, data.trg = data.src.to(device), data.trg.to(device)

        batch_size = data.trg.shape[1]

        h, h_n = enc(data.src)
        #h.shape = [src_len, batch_size, hidden_size*2]
        #h_n.shape = [1, batch_size, hidden_size]

        s_prev = h_n 

        prev_output = data.trg[0].to(device) #prev_output.shape = [1, batch_size]

        predictions = torch.zeros(1, prev_output.shape[0], len(trg_field.vocab) )
        predictions = predictions.to(device)
        
        target_sentence = []
        predicted_sentence = []

        epoch_loss = 0
        e = 0 
        for trg_id in range(1, data.trg.shape[0]):  

            output, s = model(h, prev_output, s_prev)
            #output.shape = [1, batch_size, output_dim ]
            #s.shape  = [1, batch_size, hidden_size]

            output = output.to(device)
            s_prev = s.to(device)

            predictions = torch.cat((predictions, output))

            target = data.trg[trg_id]

            if i%100 == 0 : 

                target_sentence.append(target[0].item())

            prob = [True, False][random.random() < tf]  #probability of taking the actual target 

            if prob : 

                prev_output = target

            else: 

                prev_output = torch.argmax(nn.Softmax(dim=-1)(output.squeeze(0)), dim=1)

                prev_output = prev_output.to(device)

            if i%100 == 0 : 

                predicted_sentence.append(nn.Softmax(dim=-1)(output[:, 0, :]).argmax().item())

        if i%100 == 0 : 

            print("target:")

            for word in  target_sentence: 
                print(trg_field.vocab.itos[word], end = " " ) 

            print("")

            print("output")

            for word in predicted_sentence : 

                print(trg_field.vocab.itos[word], end = " ")

            print(" ")


        target_ = data.trg[1: ].clone().to(device)

        target_  = target_.view([-1])

        target_ = list(target_)

        target_ = torch.LongTensor(target_).to(device)

        predictions = predictions[1: ]

        batch_loss = criterion(predictions.view([-1, output_dim]), target_)

        if i%100 == 0: 

            print("Batch_loss (batch)", i, "=", batch_loss)


        batch_loss.backward()
        optimizer.step()

        epoch_loss += batch_loss
        e += 1
        
        
    return e, epoch_loss 


        

In [14]:
import torch

a = torch.zeros([1, 2, 5])
torch.nn.Softmax(dim=-1)(a.squeeze(0))

tensor([[0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000]])

In [15]:
len(src_field.vocab)

7855

In [16]:

hidden_size = 300


enc = Encoder_RNN(src_dim = len(src_field.vocab) , 
                  no_features_enc = 150, 
                  hidden_size = hidden_size,
                  no_layers_enc= 2,
                  dropout=0.5)

dec = Decoder_RNN(hidden_size=hidden_size,
                  dropout = 0.5, 
                  no_features_dec=150, 
                  output_dim= len(trg_field.vocab))

enc.to(device)
dec.to(device)

att = Attention(hidden_size = hidden_size)

att.to(device)

Attention(
  (linear_enc): Linear(in_features=600, out_features=300, bias=False)
  (linear_dec): Linear(in_features=300, out_features=300, bias=False)
  (linear_sum): Linear(in_features=300, out_features=1, bias=False)
  (softmax): Softmax(dim=0)
)

In [17]:
model = Seq_2_Seq_att(dec= dec, 
                      att= att)

model.to(device)

Seq_2_Seq_att(
  (dec): Decoder_RNN(
    (embed): Embedding(5893, 150)
    (recurrent): GRU(750, 300)
    (linear): Linear(in_features=300, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (att): Attention(
    (linear_enc): Linear(in_features=600, out_features=300, bias=False)
    (linear_dec): Linear(in_features=300, out_features=300, bias=False)
    (linear_sum): Linear(in_features=300, out_features=1, bias=False)
    (softmax): Softmax(dim=0)
  )
)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(enc)+count_parameters(model)} trainable parameters')

The model has 7670593 trainable parameters


In [19]:
PAD_IDX = trg_field.vocab["<pad>"]

criterion = nn.CrossEntropyLoss(ignore_index= PAD_IDX)

learning_rate = 0.00001

optimizer = torch.optim.Adam([{"params": model.parameters()}, 
                              {"params": enc.parameters()}], lr = learning_rate)

In [21]:
EPOCH = 10
for e in range(EPOCH):

    iters , loss =  train(iterator=train_iterator, 
                  model=model, 
                  enc = enc, 
                  criterion=criterion, 
                tf = 0.75)
    
    print("EPOCH LOSS for epoch", e, "=", loss/iters)

target:
a man doing a trick on his bmx bike on top of a half pipe <trg_eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
output
a man is a a a a a a a <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos> <trg_eos>  
Batch_loss (batch) 0 = tensor(4.9415, device='cuda:0', grad_fn=<NllLossBackward>)
target:
a teenage boy wearing a blue and white uniform playing a form of hockey . <trg_eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
output
a man is is a a a a . <trg_eos> . <trg_eos> . . <trg_eos> . <trg_eos> . <trg_eos> . . <trg_eos> . <trg_eos> . <trg_eos>  
Batch_loss (batch) 100 = tensor(5.0380, device='cuda:0', grad_fn=<NllLossBackward>)
target:
a black dog is chasing another black dog that 's carrying a ball in its mouth . <trg_eos> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 
output
a man in a a a a a a a . . . . . . . <trg_eos